In [1]:
#!pip install streamlit
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import LinearRegression
import xlsxwriter

# Clustering
from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_samples, silhouette_score
from operator import itemgetter
# !pip install pyclustering
from pyclustering.cluster.kmeans import kmeans
from pyclustering.utils.metric import type_metric, distance_metric
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from scipy.spatial import distance_matrix
from scipy.spatial import distance

# Principal Components Analysis
from scipy import stats
# from sklearn.decomposition import PCA

# Classification
from sklearn.model_selection import train_test_split
import itertools

## Import Data

In [2]:
# Import Data
df = pd.read_csv('..\Test-Data\dgn_raw_data.csv')

# Add very small random number to Rating
df['target']=df['Rating'].apply(lambda x: x+random.random()/1000)

In [3]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_multiple.xlsx', engine='xlsxwriter')

## Regressions for Each UID

In [6]:
# Unique IDs
ids = df.UID.unique()

# Run linear regressions for each UID
op = pd.DataFrame
intercept = []
coefficients=[]
UID = []
for p in ids:
    df_i = df[df.UID == p]              # Create dataframe for current user id
    X = df_i.filter(regex='^[a-zA-Z][0-9]')  # df input variables only
    y = df_i['target']                  # Series of target variable
    reg = LinearRegression().fit(X, y)  # Fit linear regression
    reg.score(X, y)                     # Score regression model
    unique_id=df_i['UID'].unique()      # Saves current user id
    const = reg.intercept_              # Save intercept of the regression model
    coef = reg.coef_                    # Coefficients of regression model
    UID.append(unique_id)               # Append current user id
    intercept.append(const)             # Append current intercept
    coefficients.append(coef)           # Append current regression coefficients

# Convert newly created lists into dataframes
intercep_new = pd.DataFrame(intercept)
coefficients_new = pd.DataFrame(coefficients)
UID_new = pd.DataFrame(UID)

# Get columns names
colNames = df.drop(['Rating', 'target',], axis=1).columns
colNames = colNames.insert(1, 'Const')
colNames

# Concatenate the new dataframes and add column names
op = pd.concat([UID_new,intercep_new, coefficients_new], axis=1)
op.columns = colNames

# Save only regression coefficients for clustering
scores = op.drop(['UID','Const'], axis=1)

In [7]:
def pearson_dist(x, y):
    r = stats.pearsonr(x, y)[0]
    return (1 - r) / 2

## Cluster on Regression Coefficients

In [8]:
# Create dataframe for storing all cluster/variable combo averages and stdevs
cls_averages_all = pd.DataFrame()

# All maps for all cluster solutions
all_maps = []


####################################################

# Holds only final cluster solutions
cluster_solutions = {}

max_clusters = 6

for n in range(2, max_clusters+1):

    # change your df to numpy arr
    sample = scores.to_numpy()
    
    # define a custom metric
    metric = distance_metric(type_metric.USER_DEFINED, func=pearson_dist)
    
    # carry out a km++ init
    initial_centers = kmeans_plusplus_initializer(sample, n, random_state=123).initialize()
    
    # execute kmeans
    kmeans_instance = kmeans(sample, initial_centers, metric=metric)
    
    # run cluster analysis
    kmeans_instance.process()
    
    # get clusters
    clusters = kmeans_instance.get_clusters()
    
    # Empty dataframe to take in cluster assignments for each loop iteration
    df_clusters = pd.DataFrame()

    for i in range(len(clusters)):
        df_scores = scores.iloc[clusters[i],:]
        df[f'Optimal {n} cluster solution'] = i+1
        df_clusters = pd.concat([df_clusters, df_scores])
        df_clusters.sort_index(inplace=True)
    
    cluster_solutions[f'Optimal {n} cluster solution'] = df_clusters.iloc[:, -1]

all_cluster_solutions = pd.DataFrame.from_dict(cluster_solutions)

op = op.merge(all_cluster_solutions, left_index=True, right_index=True)

####################################################

In [9]:
#**********************************************************************#
# This is where the classification stuff begins

# Column of cluster solutions
last_var = op.shape[1]-max_clusters+1   # 18
last_cluster = op.shape[1]  # 23

In [10]:
#**********************************************************************#
# Average and Standard Deviations for each cluster/variable combination
# For cluster 1 of 2, calculate the average and stdev for each variable
# For cluster 2 of 2, calculate the average and stdev for each variable
# Etc.

for i in range(last_var, last_cluster):
    
    df_cl = op.iloc[:,np.r_[2:last_var,i]]  # i is the current cluster solution
    df_cl_cons = op.iloc[:,np.r_[1:last_var,i]]  # Same as df_cl but with constant

    if n == max_clusters:

        cls_avg_list = []

        # Take the mean of every variable for each cluster
        for k in range(1, int(df_cl_cons.iloc[:,-1].max()+1)):
            cls_mean = df_cl_cons[df_cl_cons.iloc[:,-1] == k].iloc[:,0:-1].mean()
            cls_mean = cls_mean.append(pd.Series({"Count":df_cl[df_cl.iloc[:,-1] == k].iloc[:,0:-1].shape[0]}))
            cls_avg_list.append(cls_mean)
            cls_std = df_cl_cons[df_cl_cons.iloc[:,-1] == k].iloc[:,0:-1].std()
            cls_avg_list.append(cls_std)
            # NaN means there is either only 1 observation in that cluster or none.

        # Convert to dataframe and transpose
        cls_averages = pd.DataFrame(cls_avg_list)
        cls_averages = cls_averages.T

        # Create helpful column names (Cluster # of total_#)
        col_names = []

        for col in range(1, k+1):
            new_name1 = f"Avg cluster {col}/{k}"
            col_names.append(new_name1)
            new_name2 = f"Std cluster {col}/{k}"
            col_names.append(new_name2)            

        # Rename columns
        cls_averages.columns = col_names

        cls_averages_all = pd.concat([cls_averages_all, cls_averages], axis=1)

op.to_excel(writer, index=False, sheet_name="All Regressions, Clusters")


# Add averages for all observations to cls_averages_all before exporting
all_obs = []

# Variable means for all observations
all_obs_mean = list(op.filter(regex='^[a-zA-Z][0-9]').mean().values)
all_obs_mean.insert(0,op['Const'].mean())
all_obs.append(all_obs_mean)

# cls_averages_all['new_col'] = pd.Series(list(op.filter(regex='^[a-zA-Z][0-9]').mean().values))

# Variable standard deviations for all observations
all_obs_std = list(op.filter(regex='^[a-zA-Z][0-9]').std().values)
all_obs_std.insert(0,op['Const'].std())
all_obs.append(all_obs_std)


# Save as dataframe and append to all cls_averages_all dataframe
all_obs_cols = list(op.filter(regex='^[a-zA-Z][0-9]').columns)
all_obs_cols.insert(0, "Const")
all_obs_df = pd.DataFrame(all_obs, columns=all_obs_cols)
all_obs_df = all_obs_df.T
all_obs_cols = ['All obs avg', 'All obs stdev']
all_obs_df.columns = all_obs_cols
cls_averages_all = pd.concat([cls_averages_all, all_obs_df], axis=1)


cls_averages_all.to_excel(writer, sheet_name="Cluster Avgs and StDevs")

In [11]:
writer.save()

## End of script.